# Лабораторная работа 3

## Cython

In [1]:
%load_ext Cython

In [2]:
%%cython

from cython.parallel import prange # параллельный range
from libc.math cimport sin # функция стандартной библиотеки C


ctypedef double (*func)(double x) nogil # объявление типа функции (аргумент f функции integrate)
# nogil - выключен GIL, т.к. из integrate - nogil-функции - нельзя вызвать функцию с включённым GIL


cdef double integrate(func f, float a, float b, int n_iter = 1000) nogil:
    # GIL выключен, вычисление производится параллельно
    # тип double - для повышения точности вычисления
    # f - Cython-функция, т.к. из Cython-функции нельзя вызвать функцию Python
    
    # значение интеграла на интервале, имеющем длину 0, равно 0
    if a == b:
        return 0

    # вычисление по методу трапеций
    cdef double s = (f(a) + f(b)) / 2
    cdef double h = (b - a) / n_iter
    cdef double x = a + h
    
    cdef int i
    for i in prange(n_iter - 1):
        s += f(x + i * h)
        
    cdef double result = h * s
    
    return result


cpdef integrate_function(a, b, n_iter):
    # функцию cpdef можно вызвать из функций Python и Cython
    # сделана для timeit
    # нельзя вынести в другую ячейку, т.к. функцию Cython можно вызвать только в той же ячейке
    return integrate(sin, a, b, n_iter)


In [3]:
# проверка
integrate_function(1, 10, n_iter=10**6)

1.3793736617540613

## Joblib

In [4]:
# функция из ЛР 1
def integrate(f, a: float, b: float, *, n_iter: int = 1000):
    if a == b:
        return 0

    # вычисление по методу трапеций
    h = (b - a) / n_iter
    s = (f(a) + f(b)) / 2
    x = a + h
    
    for i in range(int(n_iter)-1):
        s += f(x + i*h)
    
    result = h * s
    return float(round(result, 8))

In [5]:
from joblib import Parallel, delayed

def integrate_async(f, a: float, b: float, *, n_jobs: int = 2, n_iter: int = 1000, backend='threading'):
    step = (b - a) / n_jobs # шаг вычисления интеграла, кол-во частей равно кол-ву потоков

    with Parallel(n_jobs=n_jobs, backend=backend) as p:
        # каждый поток вычисляет значение интеграла на части интервала
        # кол-во итераций n_iter делится на кол-во потоков
        fs = (delayed(integrate)(f, a + i * step, a + (i + 1) * step, n_iter=n_iter // n_jobs)
              for i in range(n_jobs))
        return sum(p(fs)) # значение интеграла - сумма значений на каждом интервале, вычисленном в отдельном потоке


In [6]:
# проверка
import math

integrate_async(math.sin, 1, 10, n_iter=10**6)

1.37937383